In [1]:
import numpy as np

n = 9
input = open('inputs/day20.txt', 'r').read().split('\n\n')
tiles = {}
for t in input:
    id = int(t.split('\n')[0].split(' ')[1][:-1])
    tile = np.array(np.zeros((n+1,n+1), int))
    for y, l in enumerate(t.split('\n')[1:]):
        for x, c in enumerate(l):
            tile[y][x] = 0 if c == '.' else 1
    tiles[id] = tile

In [2]:
def edges(tile):
    out = []
    out.append(tile[:,0])
    out.append(tile[:,n])
    out.append(tile[0,:])
    out.append(tile[n,:])
    out.append(np.flip(tile[:,0]))
    out.append(np.flip(tile[:,n]))
    out.append(np.flip(tile[0,:]))
    out.append(np.flip(tile[n,:]))
    return out

In [3]:
all_edges = []
for id, tile in tiles.items():
    for edge in edges(tile):
        all_edges.append((id, edge))
        
n_unique = dict()
for id in tiles:
    n_unique[id] = 0

for a in range(len(all_edges)):
    unique = True
    for b in range(len(all_edges)):
        if a == b:
            continue
        if np.array_equal(all_edges[a][1], all_edges[b][1]):
            unique = False
    if unique:
        n_unique[all_edges[a][0]] += 1

p1 = 1
for id, adj in n_unique.items():
    if adj == 4:
        p1 *= id
        seed = id
p1

140656720229539

In [4]:
def find_match(edge_id, left):
    if left:
        edge = tiles[edge_id][:,n]
    else:
        edge = tiles[edge_id][n,:]
    for id, tile in tiles.items():
        if id in placed:
            continue
        if left:
            for rot in range(4):
                rotated = np.rot90(tile, k=rot)
                if np.array_equal(edge, rotated[:,0]):
                    tiles[id] = rotated
                    return id
            tile_f = np.flip(tile, 1)
            for rot in range(4):
                rotated = np.rot90(tile_f, k=rot)
                if np.array_equal(edge, rotated[:,0]):
                    tiles[id] = rotated
                    return id
        else:
            for rot in range(4):
                rotated = np.rot90(tile, k=rot)
                if np.array_equal(edge, rotated[0,:]):
                    tiles[id] = rotated
                    return id
            tile_f = np.flip(tile, 0)
            for rot in range(4):
                rotated = np.rot90(tile_f, k=rot)
                if np.array_equal(edge, rotated[0,:]):
                    tiles[id] = rotated
                    return id

In [5]:
placed = set()
id_grid = np.zeros((12,12), int)
id_grid[0][0] = seed
placed.add(seed)
tiles[seed] = np.rot90(tiles[seed], k=1)
for x in range(11):
    next_tile = find_match(id_grid[0][x], True)
    id_grid[0][x+1] = next_tile
    placed.add(next_tile)
for y in range(11):
    for x in range(12):
        next_tile = find_match(id_grid[y][x], False)
        id_grid[y+1][x] = next_tile
        placed.add(next_tile)
        
grid = np.zeros((8*12, 8*12), int)
n = 8
for y in range(12):
    for x in range(12):
        grid[y*(n):y*(n)+n,x*(n):x*(n)+n] = tiles[id_grid[y][x]][1:n+1,1:n+1]

In [6]:
monster_in = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
monster = []
for l in monster_in.split('\n'):
    row = []
    for c in l:
        row.append(1 if c == '#' else 0)
    monster.append(row)
monster = np.array(monster)

grid_rot = np.rot90(grid, k=2)
monsters = 0
for y in range(0, len(grid)-len(monster)):
    for x in range(0, len(grid[0]) - len(monster[0])):
        found = True
        for dy in range(0, len(monster)):
            for dx in range(0, len(monster[0])):
                if monster[dy][dx] == 1 and grid_rot[y+dy][x+dx] == 0:
                    found = False
        if(found):
            monsters += 1
            
grid.sum() - monsters * monster.sum()

1885